<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


## Markov Decision Process (30 Points)

HW3 :: Practical Question 2

<br>

Corresponding TA: Aryan Ahadinia, Saeed Hedayatiyan


In this question, We're going to implement a generic MDP solver, so we can solve every question by passing transition and reward functions. We use an example to test our implementation but it's important to write MDP solver functions in a generic form. So it's important to maintain notebook structure and only write codes and comment in annotated areas.

### Modeling a Problem (5 Points)

In a TV quiz show, there are several levels. At each level, if the participant answers the question correctly, they will receive some prize. If the participant's answer is wrong, they leave the competition empty-handed. Before each stage begins, the participant can decide whether to continue or withdraw and leave the game with the reward which they already earned.

Beside states representing each level, There are three terminal states of Win, Lost and Quit in the game. Actions in each state are quit and play. The player will go to the quit state with probability of 1 if they decide to take action quit. otherwise they will pass the level i by probability of win_ratio[i] and go to the state which represents the next level.

So the play / quit decision problem can be modeled as an MDP as below.

<p align="center">
<image src="./assets/quiz_problem.png">
</p>

Considering $100, $200, $300, $400 and $500 as rewards and 0.9, 0.7, 0.6, 0.3, 0.1 as win ratio for levels 0 to 4 respectively.

Complete two cells below to model the problem as an MDP. Write levels and actions as an str.


In [29]:
quiz_levels = [
    ####### Complete this list (0.25 Point) #######
    "0", "1", "2", "3", "4"
]
quiz_terminals = [
    ####### Complete this list (0.25 Point) #######
    "Lost", "Quit", "Win"
]

quiz_states = quiz_levels + quiz_terminals
quiz_actions = [
    ####### Complete this list (0.5 Point) #######
    "play", "quit"
]



In [30]:
from typing import Callable
from typing import List
from typing import Tuple
from typing import Union

In [31]:
def quiz_transition(state: str, action: str, next_state: str) -> float:
    ###########################################
    ########## Code Here! (2 Points) ##########
    ###########################################
    if action == "quit":
        if next_state == "Quit":
            return 1
        else:
            return 0

    if state == "0":
        if next_state == "1":
            return 0.9
        elif next_state == "Lost":
            return 0.1

    if state == "1":
        if next_state == "2":
            return 0.7
        elif next_state == "Lost":
            return 0.3

    if state == "2":
        if next_state == "3":
            return 0.6
        elif next_state == "Lost":
            return 0.4

    if state == "3":
        if next_state == "4":
            return 0.3
        elif next_state == "Lost":
            return 0.7

    if state == "4":
        if next_state == "Win":
            return 0.1
        elif next_state == "Lost":
            return 0.9

    return 0


def quiz_reward(state: str, action: str, next_state: str) -> Union[float, int]:
    ###########################################
    ########## Code Here! (2 Points) ##########
    ###########################################
    if action == "quit":
        if next_state == "Quit":
            return 0
        else:
            return 0

    if state == "0":
        if next_state == "1":
            return 100
        elif next_state == "Lost":
            return 0

    if state == "1":
        if next_state == "2":
            return 200
        elif next_state == "Lost":
            return -100

    if state == "2":
        if next_state == "3":
            return 300
        elif next_state == "Lost":
            return -300

    if state == "3":
        if next_state == "4":
            return 400
        elif next_state == "Lost":
            return -600

    if state == "4":
        if next_state == "Win":
            return 500
        elif next_state == "Lost":
            return -1000

    return 0


### Value Iteration (10 Points)

Now we want to implement a function which does the value iteration process. Consider that this function must be a generic function to all MDP problems. So any internal reference to global variables is forbidden.

This function gets transition function, reward function, discounting factor (gamma), list of states, list of terminal states, list of all possible actions and current value of each state as input and must return two lists: (1) updated values of each state and (2) list of best action which can be taken in each state regarding current values.


In [32]:
def mdp_iterate(
    transition_function: Callable[[str, str, str], float],
    reward_function: Callable[[str, str, str], float],
    gamma: float,
    states: List[str],
    terminals: List[str],
    actions: List[str],
    current_values: List[Union[float, int]],
) -> Tuple[List[Union[float, int]]]:
    new_values = []
    best_actions = []
    ###########################################
    ########## Code Here! (10 Points) #########
    ###########################################
    
    for state in states:
        if state in terminals:
            new_values.append(0)
            continue
        v_s = 0
        best_action_value = -100000
        best_action = 0
        for action in actions:
            action_value = 0
            for i in range(len(states)):
                new_state = states[i]
                p = transition_function(state, action, new_state)
                r = reward_function(state, action, new_state)
                action_value = action_value + p * (current_values[i] + (1 / gamma) ** i * r)
            if action_value >= best_action_value:
                best_action_value = action_value
                best_action = action

        new_values.append(v_s + best_action_value)
        best_actions.append(best_action)



    return new_values, best_actions


### MDP Solving (5 Points)

Now write a function to use the iterative approach to calculate values of each state and solve the MDP problem.

This function gets transition function, reward function, discounting factor (gamma), list of states, list of terminal states, list of all possible actions and number of iterations as input and must return a dictionary which maps states to best action can be taken. For example:

```Python
{
    'S1': 'A1',
    'S2': 'A2',
    ...
}
```


In [33]:
from typing import Dict


def mdp_solve(
    transition_function: Callable[[str, str, str], float],
    reward_function: Callable[[str, str, str], float],
    gamma: float,
    states: List[str],
    terminals: List[str],
    actions: List[str],
    iter: int,
) -> Dict['str', 'str']:
    ###########################################
    ########## Code Here! (5 Points) #########
    ###########################################
    cur_values = [0 for i in range(len(states))]
    best = dict()
    for i in range(iter):
        cur_values, actions = mdp_iterate(transition_function, reward_function, gamma, states, terminals, actions, cur_values)
    for i in range(len(actions)):
        if states[i] in terminals:
            continue
        best.update({states[i]: actions[i]})
    return best  # Change this line to return the correct value


By now, running this code must solve the problem described above.


In [34]:
mdp_solve(
    quiz_transition,
    quiz_reward,
    1,
    quiz_states,
    quiz_terminals,
    quiz_actions,
    1000,
)


{'0': 'play', '1': 'play', '2': 'play', '3': 'quit', '4': 'quit'}

### Analyzing effect of discounting factor (10 Points)

Now, solve the problem for different values of discounting factor. Then draw a plot to show how increasing discounting factor effect on best decision that should be take. Then write a paragraph about the result.

In [35]:
###########################################
########## Code Here! (5 Points) #########
###########################################
gammas = [1, 0.8, 0.5, 0.25]
for gamma in gammas:
    print("Best Policy for gamma =", gamma)
    best_actions = mdp_solve(
                    quiz_transition,
                    quiz_reward,
                    gamma,
                    quiz_states,
                    quiz_terminals,
                    quiz_actions,
                    1000,
                    )
    print(best_actions, "\n")

Best Policy for gamma = 1
{'0': 'play', '1': 'play', '2': 'play', '3': 'quit', '4': 'quit'} 

Best Policy for gamma = 0.8
{'0': 'play', '1': 'play', '2': 'quit', '3': 'quit', '4': 'quit'} 

Best Policy for gamma = 0.5
{'0': 'play', '1': 'quit', '2': 'quit', '3': 'quit', '4': 'quit'} 

Best Policy for gamma = 0.25
{'0': 'play', '1': 'quit', '2': 'quit', '3': 'quit', '4': 'quit'} 



######### Write Here (5 Points) ########### </br>
As we see above, the less $\gamma$ becomes, the sooner we quit the game; because by decreasing $\gamma$, we essentially decrease the the reward in later steps, so expected reward of quitting sooner becomes more promissing. In fact, by decreasing $\gamma$, we choose a low-risk low-reward strategy.
For $\gamma = 1$, we quit at the 3rd level, for $\gamma = 0.8$, we quit at the 2nd level and for $\gamma = 0.5$, we quit at the 1st level.